# Sequences
- at most, an immutable sequence type should support two things.
    1. return the length of the sequence.
    2. given an index, returning the element at that index.

In [3]:
[1, 2, 3].__getitem__(1)

2

In [4]:
[1, 2, 3].__getitem__(3)

IndexError: list index out of range

## lets implement our own __getitem__

In [5]:
def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [7]:
from time import perf_counter

start = perf_counter()
print(fib(35))
print(f'Elapsed: {perf_counter() - start}s') 

14930352
Elapsed: 2.1503845839979476s


In [9]:
from functools import lru_cache

@lru_cache
def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

start = perf_counter()
print(fib(350))
print(f'Elapsed: {perf_counter() - start}s') 

10119911756749018713965376799211044556615579094364594923736162239653346274
Elapsed: 0.00042183299956377596s


In [42]:
class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n

    def __getitem__(self, s):
        if isinstance(s, int):
            if s < 0 or s > self.n:
                raise IndexError
            else:
                return Fib._fib(s)

    @staticmethod
    @lru_cache
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)


In [43]:
f = Fib(10)

### now we can iterate over f in any method we want

In [44]:
f.__getitem__(5)

8

In [45]:
list(f)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

In [46]:
index = 0
while True:
    try:
        print(f.__getitem__(index))
    except IndexError:
        break
    index+=1
    

1
1
2
3
5
8
13
21
34
55
89


In [47]:
for item in f:
    print(item)

1
1
2
3
5
8
13
21
34
55
89


In [48]:
[x for x in f]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

In [49]:
f[5]

8

In [50]:
f[-1]

IndexError: 

### lets implement negative sequence

In [51]:
class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n

    def __getitem__(self, s):
        if isinstance(s, int):
            if s < 0:
                s = self.n + s
            if s < 0 or s > self.n:
                raise IndexError
            else:
                return Fib._fib(s)

    @staticmethod
    @lru_cache
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)


In [52]:
f = Fib(10)
f[-1]

55

In [53]:
fib[0:5]

TypeError: 'functools._lru_cache_wrapper' object is not subscriptable

In [66]:
s = slice(0, 5)
type(s), s.indices(10)

(slice, (0, 5, 1))

In [70]:
s = slice(-1, -4, -1)
type(s), s.indices(10)

(slice, (9, 6, -1))

In [75]:
list(range(9, 6, -1))

[9, 8, 7]

### that is what we want

In [101]:
class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n

    def __getitem__(self, s):
        if isinstance(s, int):
            if s < 0:
                s = self.n + s + 1
            if s < 0 or s > self.n:
                raise IndexError
            else:
                return Fib._fib(s)
        else:
            start, stop, step = s.indices(self.n + 1)
            rng = range(start, stop, step)
            return [Fib._fib(i) for i in rng]

    @staticmethod
    @lru_cache
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)


In [102]:
f = Fib(10)
f[10], f[-1]

(89, 89)

In [103]:
list(f)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

In [104]:
f[-1:-4:-1]

[89, 55, 34]

# where named tupple fails us

In [105]:
from collections import namedtuple

In [108]:
Point2D = namedtuple('Point2D', ['x', 'y'])
pt1 = Point2D(10, 20)
pt1

Point2D(x=10, y=20)

In [117]:
pt1 = pt1._replace(x = 'abc')
pt1

Point2D(x='abc', y=20)

In [119]:
pt1 = Point2D('abc', 2+5j)
pt1

Point2D(x='abc', y=(2+5j))

# if i want to co-ords to be integers only 

In [122]:
import numbers

In [129]:
isinstance(10, numbers.Number), isinstance('abc', numbers.Number), isinstance(2J, numbers.Number)

(True, False, True)

In [130]:
isinstance(10, numbers.Real), isinstance('abc', numbers.Real), isinstance(2J, numbers.Real)

(True, False, False)

In [133]:
class Point:
    def __init__(self, x, y):
        if isinstance(x, numbers.Real) and isinstance(y, numbers.Real):
            self._pt = (x, y)
        else:
            raise TypeError('Point coords must be real nums')

    def __repr__(self):
        return f'Point(x={self._pt[0]}, y={self._pt[1]})'        

In [134]:
pt1 = Point(10, 20)
pt1

Point(x=10, y=20)

In [135]:
x, y = pt1

TypeError: cannot unpack non-iterable Point object

# lets make our class an iterable

In [137]:
class Point:
    def __init__(self, x, y):
        if isinstance(x, numbers.Real) and isinstance(y, numbers.Real):
            self._pt = (x, y)
        else:
            raise TypeError('Point coords must be real nums')

    def __repr__(self):
        return f'Point(x={self._pt[0]}, y={self._pt[1]})'  

    def __len__(self):
        return len(self._pt)

    def __getitem__(self, s):
        # i am delegating if s is slice or int to tuple object in _pt
        return self._pt[s]

In [138]:
pt1 = Point(10, 20)
pt1

Point(x=10, y=20)

In [139]:
x, y = pt1
x, y

(10, 20)

In [140]:
list(pt1)

[10, 20]

### 🧩 1. How does `list(f)` know to call `__getitem__`?

When you do:

```python
f = Fib(10)
list(f)
```

Python checks whether `f` is **iterable** — that is, whether it can be looped over.

It tries the following, in order:

1. **Check for an `__iter__` method.**

   * If present, it calls `iter(f)`, which should return an *iterator* — an object with a `__next__` method.
   * Then, it repeatedly calls `next(iterator)` until `StopIteration` is raised.

2. **If there’s no `__iter__`, fall back to the old-style sequence protocol**:

   * Python will repeatedly call `f[0]`, `f[1]`, `f[2]`, …
   * until you raise an `IndexError`.

So in your case:

* You haven’t defined `__iter__`.
* Python therefore uses the old sequence protocol.
* It calls `f[0]`, then `f[1]`, then `f[2]`, … until it hits `IndexError`.

That’s why defining `__getitem__` is enough to make your object iterable.

---

### 🔄 2. Difference between `__getitem__` and `__next__`

| Aspect                 | `__getitem__`                                                 | `__next__`                                             |
| :--------------------- | :------------------------------------------------------------ | :----------------------------------------------------- |
| Belongs to             | **Sequence protocol**                                         | **Iterator protocol**                                  |
| Usage                  | Defines how to access elements by index (`f[i]`)              | Defines how to get the *next* element in a stream      |
| Iteration style        | Python loops calling `obj[0]`, `obj[1]`, … until `IndexError` | Python loops calling `next(obj)` until `StopIteration` |
| Needs companion method | Usually used with `__len__` (optional)                        | Must have `__iter__` that returns `self`               |
| Typical use case       | For “indexable” containers (like lists, tuples, strings)      | For generators and streaming data                      |

Example of an **iterator-style Fib**:

```python
class FibIter:
    def __init__(self, n):
        self.n = n
        self.a, self.b = 0, 1
        self.count = 0

    def __iter__(self):
        return self  # iterator returns itself

    def __next__(self):
        if self.count >= self.n:
            raise StopIteration
        self.a, self.b = self.b, self.a + self.b
        self.count += 1
        return self.a
```

Now:

```python
for x in FibIter(5):
    print(x)
```

will call `__next__()` repeatedly.

---

### 🧠 Summary

* `__getitem__` → sequence-style iteration (old-school, index-based).
* `__next__` + `__iter__` → iterator-style iteration (modern, preferred).
* Python automatically falls back to `__getitem__` if `__iter__` is missing.
